In [3]:
#importing other required libraries
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from tensorflow.keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
# from keras.layers import Flatten, Dense, BatchNormalization, Activation,
# Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import random

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
!unzip /content/gdrive/MyDrive/PCOS_images.zip -d PCOS_images

Archive:  /content/gdrive/MyDrive/PCOS_images.zip
   creating: PCOS_images/test/infected/
  inflating: PCOS_images/test/infected/img_0_4927.jpg  
  inflating: PCOS_images/test/infected/img_0_4969.jpg  
  inflating: PCOS_images/test/infected/img_0_4974.jpg  
  inflating: PCOS_images/test/infected/img_0_4978.jpg  
  inflating: PCOS_images/test/infected/img_0_4981.jpg  
  inflating: PCOS_images/test/infected/img_0_5011.jpg  
  inflating: PCOS_images/test/infected/img_0_5019.jpg  
  inflating: PCOS_images/test/infected/img_0_5036.jpg  
  inflating: PCOS_images/test/infected/img_0_5078.jpg  
  inflating: PCOS_images/test/infected/img_0_5091.jpg  
  inflating: PCOS_images/test/infected/img_0_5103.jpg  
  inflating: PCOS_images/test/infected/img_0_5125.jpg  
  inflating: PCOS_images/test/infected/img_0_5154.jpg  
  inflating: PCOS_images/test/infected/img_0_5157.jpg  
  inflating: PCOS_images/test/infected/img_0_5164.jpg  
  inflating: PCOS_images/test/infected/img_0_5171.jpg  
  inflating: P

In [11]:
datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, shear_range=0.2, zoom_range=0.2,rotation_range = 20,fill_mode = 'nearest')
testgen = ImageDataGenerator(rescale=1./255, horizontal_flip=False)
train_data=datagen.flow_from_directory('PCOS_images/train', target_size=(224,224), batch_size=32,class_mode='binary')
val_data=datagen.flow_from_directory('PCOS_images/validate', target_size=(224,224), batch_size=32,class_mode='binary')
test_data=testgen.flow_from_directory('PCOS_images/test', target_size=(224,224), batch_size=32, class_mode='binary')

Found 1924 images belonging to 2 classes.
Found 974 images belonging to 2 classes.
Found 948 images belonging to 2 classes.


In [6]:
from keras.layers.core.flatten import Flatten
model = Sequential()
model.add(tf.keras.layers.Conv2D(filters = 8, kernel_size = 3, input_shape = [224, 224, 3], activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
model.add(tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
model.add(Flatten())
model.add(tf.keras.layers.Dense(30, activation = 'relu'))
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [7]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 8)       224       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 112, 112, 8)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 16)      1168      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 56, 56, 16)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 56, 56, 32)        4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 28, 28, 32)      

In [8]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
model.fit(train_data, validation_data=val_data, epochs = 10, batch_size=32, verbose = 1)

Epoch 1/10
61/61 [==============================] - 106s 2s/step - loss: 0.5285 - accuracy: 0.7812 - val_loss: 0.3934 - val_accuracy: 0.8316
Epoch 2/10
61/61 [==============================] - 103s 2s/step - loss: 0.3440 - accuracy: 0.8508 - val_loss: 0.4794 - val_accuracy: 0.8285
Epoch 3/10
61/61 [==============================] - 105s 2s/step - loss: 0.2951 - accuracy: 0.8784 - val_loss: 0.2436 - val_accuracy: 0.9004
Epoch 4/10
61/61 [==============================] - 103s 2s/step - loss: 0.2760 - accuracy: 0.8794 - val_loss: 0.2180 - val_accuracy: 0.9076
Epoch 5/10
61/61 [==============================] - 106s 2s/step - loss: 0.2517 - accuracy: 0.8820 - val_loss: 0.2459 - val_accuracy: 0.8932
Epoch 6/10
61/61 [==============================] - 101s 2s/step - loss: 0.1970 - accuracy: 0.9200 - val_loss: 0.2405 - val_accuracy: 0.9066
Epoch 7/10
61/61 [==============================] - 105s 2s/step - loss: 0.1473 - accuracy: 0.9465 - val_loss: 0.2379 - val_accuracy: 0.8943
Epoch 8/10
61

In [20]:
model.evaluate(val_data)

31/31 [==============================] - 24s 779ms/step - loss: 0.0295 - accuracy: 0.9918


[0.02951646037399769, 0.9917864203453064]

In [16]:
test_data.class_indices

{'infected': 0, 'notinfected': 1}

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [23]:
from tensorflow.keras.preprocessing import image

In [27]:
# Image of friend of mine who had PCOS
test_img = image.load_img('/content/photo1660232467.jpeg', target_size = (224, 224))

In [28]:
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img, axis = 0)
result = model.predict(test_img)
print(result)

[[0.]]


In [31]:
test_data.class_indices

{'infected': 0, 'notinfected': 1}

In [32]:
if result[0][0] == 0:
  print("Positive for PCOS")
else:
  print("Negative for PCOS")

Positive for PCOS
